In [173]:
try:
    import numpy as np
except ImportError:
    !python -m pip install numpy
try:
    import pandas as pd
except ImportError:
    !python -m pip install pandas


In [174]:
import pandas as pd
import random

In [175]:
file_path = '..\\Data\\df_most_recent.csv'
df = pd.read_csv(file_path)
print(df.head(5))
print(len(df))

       open_time  open_price  high_price  low_price  close_price   volume  \
0  1713587100000     63845.9     63933.3    63831.7      63921.3  140.680   
1  1713587400000     63921.3     63939.5    63836.3      63932.1  312.187   
2  1713587700000     63932.1     64127.9    63932.0      64100.1  516.926   
3  1713588000000     64100.1     64206.8    64060.0      64159.1  448.984   
4  1713588300000     64159.1     64257.3    64129.1      64175.1  538.741   

       turnover  
0  8.986484e+06  
1  1.994644e+07  
2  3.310749e+07  
3  2.879849e+07  
4  3.458664e+07  
3000


In [176]:
class GapStrategy:
    def __init__(self):
        self.long_gaps = []
        self.short_gaps = []
        self.long_Gaps_Checked_Until = 0
        self.short_Gaps_Checked_Until = 0
        self.filled_gaps_index = 100
    
    def strategy(self,df,candle_index):
        candle = df.iloc[candle_index]
        self.remove_old_and_filled_gaps(df,candle_index)
        self.add_new_gaps(df,candle_index)

        is_gap, gap = self.if_current_candle_in_gap(df,candle_index)
        if is_gap == 1:
            gap['Position Type'] = "Long"
            return gap
        elif is_gap == 2:
            gap['Position Type'] = "Short"
            return gap
        return None
    
    def remove_old_and_filled_gaps(self,df,candle_index):
        #Starting from the oldest gap remove all gaps that are older than 14 days
        for gap in self.long_gaps:
            if candle_index - gap['end'] > 4032:
                self.long_gaps.remove(gap)
            else:
                break

        for gap in self.short_gaps:
            if candle_index - gap['end'] > 4032:
                self.short_gaps.remove(gap)
            else:
                break
        
        for self.filled_gaps_index in range(self.filled_gaps_index, candle_index + 1):
            current_candle = df.iloc[self.filled_gaps_index]

            for gap in self.long_gaps:
                if current_candle['low_price'] < gap['high_price'] and gap['end'] < self.filled_gaps_index:
                    print("Removing Gap -", gap)
                    self.long_gaps.remove(gap)
                else:
                    if gap['end'] > self.filled_gaps_index:
                        print("This Shouldn't happen: ", gap['end'], ' - ', self.filled_gaps_index)
                    break
            
            for gap in self.short_gaps:
                if current_candle['high_price'] < gap['low_price'] and gap['end'] < self.filled_gaps_index:
                    self.short_gaps.remove(gap)
                    print("Removing Gap -", gap)
                else:
                    break

    
    #Function that adds new gaps
    def add_new_gaps(self,df,candle_index):

        #Check if there are new unfilled gaps
        new_long_gaps = self.calculate_long_gaps(df,candle_index)
        for gap in new_long_gaps:
            self.long_gaps.append(gap)

        new_short_gaps = self.calculate_short_gaps(df,candle_index)
        for gap in new_short_gaps:
            self.short_gaps.append(gap)


    def calculate_long_gaps(self,df,candle_index):
        gaps = []
        for self.long_Gaps_Checked_Until in range(self.long_Gaps_Checked_Until, candle_index - 2):
            first_candle = df.iloc[self.long_Gaps_Checked_Until]
            third_candle = df.iloc[self.long_Gaps_Checked_Until + 2]

            if first_candle['high_price'] <= third_candle['low_price']:
                entry_price = (third_candle['low_price'] + first_candle['high_price']) / 2
                stop_loss = first_candle['low_price']
                stop_loss_in_perc = 1 - stop_loss/entry_price
                if(stop_loss_in_perc >= 0.002):
                    gap_info = {
                        'start': self.long_Gaps_Checked_Until,
                        'end': self.long_Gaps_Checked_Until + 2,
                        'time': first_candle['open_time'],
                        'high_price': third_candle['low_price'],
                        'low_price': first_candle['high_price'],
                        'entry_price': entry_price,
                        'stop_loss': first_candle['low_price'],
                        'stop_loss_in_perc': stop_loss_in_perc,
                        'filled': False
                        }
                    gaps.append(gap_info)
            else:
                for gap in gaps[::-1]:
                    if first_candle['low_price'] < gap['high_price']:
                        # Remove the gap from the list as it's no longer needed
                        gaps.remove(gap)

        self.long_Gaps_Checked_Until += 1
        return gaps  # Return the list of gaps for further use or analysis
    
    def calculate_short_gaps(self,df,candle_index):
        gaps = []
        for self.short_Gaps_Checked_Until in range(self.short_Gaps_Checked_Until, candle_index - 2):
            first_candle = df.iloc[self.short_Gaps_Checked_Until]
            third_candle = df.iloc[self.short_Gaps_Checked_Until + 2]

            if first_candle['low_price'] > third_candle['high_price']:
                entry_price = (first_candle['low_price'] + third_candle['high_price']) / 2
                stop_loss = first_candle['high_price']
                stop_loss_in_perc = 1 - entry_price/stop_loss
                if(stop_loss_in_perc >= 0.002):
                    gap_info = {
                        'start': self.short_Gaps_Checked_Until,
                        'end': self.short_Gaps_Checked_Until + 2,
                        'time': first_candle['open_time'],
                        'high_price': first_candle['low_price'],
                        'low_price': third_candle['high_price'],
                        'entry_price': entry_price,
                        'stop_loss': first_candle['high_price'],
                        'stop_loss_in_perc': stop_loss_in_perc,
                        'filled': False
                        }
                    gaps.append(gap_info)
            else:
                for gap in gaps[::-1]:

                    if first_candle['high_price'] > gap['low_price'] :
                        # Remove the gap from the list as it's no longer needed
                        gaps.remove(gap)
                        
        self.short_Gaps_Checked_Until += 1
        return gaps  # Return the list of gaps for further use or analysis
    
    #Function that checks whether the current candle goes into a gap and if it does returns 1 or 2 (1 for Long Gap and 2 for Short Gap)
    def if_current_candle_in_gap(self,df,candle_index):
        current_candle = df.iloc[candle_index]

        for gap in self.long_gaps[::-1]:
            if current_candle['low_price'] <= gap['entry_price']:
                self.long_gaps.remove(gap)
                return 1, gap
            
        for gap in self.short_gaps[::-1]:
            if current_candle['high_price'] >= gap['entry_price']:
                self.short_gaps.remove(gap)
                return 2, gap

        return 0, []

In [177]:
class StrategyBuilder:
    def __init__(self,df):
        self.df = df
        self.is_gap_strategy_initialized = False

    def basic_gap_strategy(self,current_index):
        if self.is_gap_strategy_initialized is False:
            self.is_gap_strategy_initialized = True
            self.gap_strategy = GapStrategy()
        return self.gap_strategy.strategy(self.df,current_index)
         
        


In [178]:
class BackTestingEnvironment: 
    def __init__(self, df,strategy, fee_rate = 0.00055):
        #Environment Variables
        self.df = df
        self.candle_index = 100
        self.strategy = strategy

        #Trade Variables
        self.position = None
        self.take_profit = None
        self.stop_loss = None
        self.accountBalance = 100
        self.entry_price = 0
        self.trade_fee = 0
        self.fee_rate = fee_rate*2
        self.trade_value = 0
        self.trade_quantity = 0
        self.trade_entry_time = 0

        #Evaluation Variables
        self.total_trades = 0
        self.total_wins = 0
        self.total_losses = 0

    #Function that starts the BackTesting
    def start(self):
        #Iteration Through the whole dataset starting from self.candle_index
        for self.candle_index in range (self.candle_index,len(self.df)):

            #If Not in a position check if criteria met for the strategy to enter a trade
            if self.position == None:
                result = self.strategy(self.candle_index)
                if(result):
                    
                    self.total_trades += 1
                    self.position = result['Position Type']

                    self.entry_price = result['entry_price']
                    self.stop_loss = result['stop_loss']
                    stop_loss_in_perc = result['stop_loss_in_perc']
                    money_risk = self.accountBalance * 0.02
                    self.trade_value = money_risk / stop_loss_in_perc
                    tp_in_perc = stop_loss_in_perc * 2

                    self.trade_quantity = self.trade_value / self.entry_price
                    self.trade_fee = self.trade_value * self.fee_rate
                    self.trade_entry_time = self.df['open_time'][self.candle_index]
                    
                    if self.position == "Long":
                        self.take_profit = self.entry_price * (1 + tp_in_perc)           
                    else:
                        self.take_profit = self.entry_price * (1 - tp_in_perc)
                    
            else:
                #If in a trade, check if the trade critera for closing the trades are met and if there are calculate the results
                current_price_high = self.df.iloc[self.candle_index]['high_price']
                current_price_low = self.df.iloc[self.candle_index]['low_price']
                if self.position == 'Long':

                    if current_price_high >= self.take_profit or current_price_low <= self.stop_loss:
                        random_number = random.random()
                        close_price = 0

                        #Since the candle low/high might hit both the TP and SP we simply make it 50/50 which one is checked first
                        if random_number >= 0.5:
                            if current_price_high >= self.take_profit:
                                close_price = self.take_profit
                            else:
                                close_price = self.stop_loss
                        else:
                            if current_price_low <= self.stop_loss:
                                close_price = self.stop_loss
                            else:
                                close_price = self.take_profit

                        reward = self.calculate_reward(close_price)

                        if reward < 0:
                            self.total_losses += 1
                        else:
                            self.total_wins +=1

                        print('Long Trade Closed')
                        self.reset_trade_variables()
                        result = None
                else:
                    if current_price_low <= self.take_profit or current_price_high >= self.stop_loss:
                        random_number = random.random()
                        close_price = 0

                        #Since the candle low/high might hit both the TP and SP we simply make it 50/50 which one is checked first
                        if random_number >= 0.5:
                            if current_price_high >= self.stop_loss:
                                close_price = self.stop_loss
                            else:
                                close_price = self.take_profit
                        else:
                            if current_price_low <= self.take_profit:
                                close_price = self.take_profit
                            else:
                                close_price = self.stop_loss

                        reward = self.calculate_reward(close_price)

                        if reward < 0:
                            self.total_losses += 1
                        else:
                            self.total_wins +=1

                        print('Short Trade Closed')
                        self.reset_trade_variables()
                        result = None


    #Function that calculates the results from a trade
    def calculate_reward(self, current_price):
            #Depending on the type of Trade - Long/Short, it calculates the results and the fees paid for the trade
            if self.position == 'Long':
                tradeResult = (current_price - self.entry_price)*self.trade_quantity - self.trade_fee
                print("Trade Fee is:",self.trade_fee)
                self.accountBalance += tradeResult

                print('Trade Type: ', self.position, ' resulted in: ', tradeResult, ' Making the account balance: ', self.accountBalance)
                return tradeResult
            elif self.position == 'Short':
                tradeResult = (self.entry_price - current_price)*self.trade_quantity - self.trade_fee
                print("Trade Fee is:",self.trade_fee)
                self.accountBalance += tradeResult

                print('Trade Type: ', self.position, ' resulted in: ', tradeResult, ' Making the account balance: ', self.accountBalance)
                return tradeResult  # Profit or loss for short position

    #Function that resets all the Variables needed for a trade
    def reset_trade_variables(self):
        self.position = None
        self.take_profit = None
        self.stop_loss = None
        self.entry_price = 0
        self.trade_value = 0
        self.trade_quantity = 0
        self.trade_entry_time = 0
                


In [179]:
strategy = StrategyBuilder(df)
BT_env =  BackTestingEnvironment(df,strategy.basic_gap_strategy)

In [180]:
BT_env.start()


Trade Fee is: 1.0812442396313446
Trade Type:  Short  resulted in:  -3.0853219223049217  Making the account balance:  96.91467807769507
Short Trade Closed
Trade Fee is: 0.3190150576985363
Trade Type:  Long  resulted in:  3.5575720654092495  Making the account balance:  100.47225014310432
Long Trade Closed
Trade Fee is: 0.4626782765850603
Trade Type:  Long  resulted in:  -2.472123279447123  Making the account balance:  98.0001268636572
Long Trade Closed
Trade Fee is: 0.7981458487098562
Trade Type:  Short  resulted in:  -2.763457211167625  Making the account balance:  95.23666965248958
Short Trade Closed
Trade Fee is: 0.7967566518311913
Trade Type:  Long  resulted in:  3.0127101342684206  Making the account balance:  98.249379786758
Long Trade Closed
Trade Fee is: 0.6331983772290464
Trade Type:  Long  resulted in:  -2.5981859729642327  Making the account balance:  95.65119381379377
Long Trade Closed
Removing Gap - {'start': 274, 'end': 276, 'time': 1713669300000.0, 'high_price': 65444.3, 

KeyboardInterrupt: 

In [164]:
print(BT_env.total_trades)
print(BT_env.total_wins)

82
22
